# Introduction

This lesson is a brief introduction to TOAST and its data representations.  This next cell is just initializing some things for the notebook.

In [ ]:
# Are you using a special reservation for a workshop?
# If so, set it here:
nersc_reservation = None

# Load common tools for all lessons
import sys
sys.path.insert(0, "..")
from lesson_tools import (
    check_nersc,
    fake_focalplane
)
nersc_host, nersc_repo = check_nersc(reservation=nersc_reservation)
if nersc_host is not None:
    %reload_ext slurm_magic

## Runtime Environment

You can get the current TOAST runtime configuration from the "Environment" class.

In [ ]:
import toast

env = toast.Environment.get()
# FIXME:  update __repr__ of Environment class
print(env)


## Data Model

Before using TOAST for simulation or analysis, it is important to discuss how data is stored in memory and how that data can be distributed among many processes to parallelize large workflows.

First, let's create a fake focalplane of detectors to use throughout this example.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Generate a fake focalplane with 7 pixels, each with 2 detectors.

fp = fake_focalplane()

# Make a plot of this focalplane layout.

detnames = list(sorted(fp.keys()))
detquat = {x: fp[x]["quat"] for x in detnames}
detfwhm = {x: fp[x]["fwhm_arcmin"] for x in detnames}
detlabels = {x: x for x in detnames}
detpolcol = {x: "red" if i % 2 == 0 else "blue" for i, x in enumerate(detnames)}

toast.tod.plot_focalplane(
    detquat, 4.0, 4.0, None, fwhm=detfwhm, polcolor=detpolcol, labels=detlabels
)


### Observations with Time Ordered Data

TOAST works with data organized into *observations*.  Each observation is independent of any other observation.  An observation consists of co-sampled detectors for some span of time.  The intrinsic detector noise is assumed to be stationary within an observation.  Typically there are other quantities which are constant for an observation (e.g. elevation, weather conditions, satellite spin axis, etc).

An observation is just a dictionary with at least one member ("tod") which is an instance of a class that derives from the `toast.TOD` base class.

The inputs to a TOD class constructor are at least:

1. The detector names for the observation.
2. The number of samples in the observation.
3. The geometric offset of the detectors from the boresight.
4. Information about how detectors and samples are distributed among processes.  More on this below.

The TOD class can act as a storage container for different "flavors" of timestreams as well as a source and sink for the observation data (with the read_\*() and write_\*() methods):

In [ ]:
import toast.qarray as qa

nsamples = 1000

obs = dict()
obs["name"] = "20191014_000"

# The type of TOD class is usually specific to the data processing job.
# For example it might be one of the simulation classes or it might be
# a class that loads experiment data.  Here we just use a simple class
# that is only used for testing and which reads / writes data to internal memory
# buffers.

tod = toast.tod.TODCache(None, detnames, nsamples, detquats=detquat)
obs["tod"] = tod

# The TOD class has methods to get information about the data:

print("TOD has detectors {}".format(", ".join(tod.detectors)))
print("TOD has {} total samples for each detector".format(tod.total_samples))

# Write some data.  Not every TOD derived class supports writing (for example,
# TOD classes that represent simulations).

t_delta = 1.0 / fp[detnames[0]]["rate"]
tod.write_times(stamps=np.arange(0.0, nsamples * t_delta, t_delta))
tod.write_boresight(
    data=qa.from_angles(
        (np.pi / 2) * np.ones(nsamples),
        (2 * np.pi / nsamples) * np.arange(nsamples),
        np.zeros(nsamples)
    )
)
for d in detnames:
    tod.write(detector=d, data=np.random.normal(scale=fp[d]["NET"], size=nsamples))
    tod.write_flags(detector=d, flags=np.zeros(nsamples, dtype=np.uint8))

# Read it back

print("TOD timestampes = {} ...".format(tod.read_times()[:5]))
print("TOD boresight = \n{} ...".format(tod.read_boresight()[:5,:]))
for d in detnames:
    print("TOD detector {} = {} ...".format(d, tod.read(detector=d, n=5)))
    print("TOD detector {} flags = {} ...".format(d, tod.read_flags(detector=d, n=5)))

# Store some data in the cache.  The "cache" member variable looks like a dictionary of
# numpy arrays, but the memory used is allocated in C, so that we can actually clear
# these buffers when needed.

for d in detnames:
    processed = tod.read(detector=d)
    processed /= 2.0
    # By convention, we usually name buffers in the cache by <prefix>_<detector>
    tod.cache.put("processed_{}".format(d), processed)
print("TOD cache now contains {} bytes".format(tod.cache.report(silent=True)))


### Comm : Groups of Processes

A toast.Comm instance takes the global number of processes available (MPI.COMM_WORLD) and divides them into groups.  Each process group is assigned one or more observations.  Since observations are independent, this means that different groups can be independently working on separate observations in parallel.  It also means that inter-process communication needed when working on a single observation can occur with a smaller set of processes.

At NERSC, this notebook is running on a login node, so we cannot use MPI.  Constructing a default `toast.Comm` whenever MPI use is disabled will just produce a single group of one process.  See the parallel example at the end of this notebook for a case with multiple groups.

In [ ]:
comm = toast.Comm()
# FIXME:  update __repr__ of Comm class
print(comm)

### Data : a Collection of Observations

A toast.Data instance is mainly just a list of observations.  However remember that each process group will have a different list.  Since we have only one group of one process, this example is not so interesting.  See the parallel case at the end of the notebook.

In [ ]:
data = toast.Data(comm)
data.obs.append(obs)

### Data Distribution

Recapping previous sections, we have some groups of processes, each of which has a set of observations.  Within a single process group, the detector data is distributed across the processes within the group.  That distribution is controlled by the size of the communicator passed to the TOD class, and also by the `detranks` parameter of the constructor.  This detranks number sets the dimension of the process grid in the detector direction.  For example, a value of "1" means that every process has all detectors for some span of time.  A value equal to the size of the communicator results in every process having some number of detectors for the entire observation.  The detranks parameter must divide evenly into the number of processes in the communicator and determines how the processes are arranged in a grid.

As a concrete example, imagine that MPI.COMM_WORLD has 24 processes.  We split this into 4 groups of 6 procesess.  There are 6 observations of varying lengths and every group has one or 2 processes.  Here is a picture of what data each process would have.  The global process number is shown as well as the rank within the group:
<img src="toast_data_dist.png">

## Utilities

## Running the Test Suite

## Running in Parallel

The NERSC login nodes do not support MPI, so all of the previous examples are running serially on a login node.  To run in parallel, we can submit a batch job or get an interactive session on one or more compute nodes.

In [ ]:
%%writefile intro.py

import toast
from toast.mpi import MPI

import numpy as np
import matplotlib.pyplot as plt

import toast.qarray as qa


# Runtime Environment

env = toast.Environment.get()
if MPI.COMM_WORLD.rank == 0:
    env.print()

# Create a fake focalplane

fp = fake_focalplane()

detnames = list(sorted(fp.keys()))
detquat = {x: fp[x]["quat"] for x in detnames}
detfwhm = {x: fp[x]["fwhm_arcmin"] for x in detnames}
detlabels = {x: x for x in detnames}
detpolcol = {x: "red" if i % 2 == 0 else "blue" for i, x in enumerate(detnames)}

if MPI.COMM_WORLD.rank == 0:
    outfile = "intro_focalplane.pdf"
    toast.tod.plot_focalplane(
        detquat, 4.0, 4.0, outfile, fwhm=detfwhm, polcolor=detpolcol, labels=detlabels
    )

# The TOD base and derived classes

nsamples = 1000

obs = dict()

# The type of TOD class is usually specific to the data processing job.
# For example it might be one of the simulation classes or it might be
# a class that loads experiment data.  Here we just use a simple class
# that is only used for testing and which reads / writes data to internal memory
# buffers.

tod = toast.tod.TODCache(MPI.COMM_WORLD, detnames, nsamples, detquats=detquat)
obs["tod"] = tod

# The TOD class has methods to get information about the data:

print("TOD has detectors {}".format(", ".join(tod.detectors)))
print("TOD has {} total samples for each detector".format(tod.total_samples))

# Write some data.  Not every TOD derived class supports writing (for example,
# TOD classes that represent simulations).

t_delta = 1.0 / fp[detnames[0]]["rate"]
tod.write_times(stamps=np.arange(0.0, nsamples * t_delta, t_delta))
tod.write_boresight(
    data=qa.from_angles(
        (np.pi / 2) * np.ones(nsamples),
        (2 * np.pi / nsamples) * np.arange(nsamples),
        np.zeros(nsamples)
    )
)
for d in detnames:
    tod.write(detector=d, data=np.random.normal(scale=fp[d]["NET"], size=nsamples))
    tod.write_flags(detector=d, flags=np.zeros(nsamples, dtype=np.uint8))

# Read it back

print("TOD timestampes = {} ...".format(tod.read_times()[:5]))
print("TOD boresight = \n{} ...".format(tod.read_boresight()[:5,:]))
for d in detnames:
    print("TOD detector {} = {} ...".format(d, tod.read(detector=d, n=5)))
    print("TOD detector {} flags = {} ...".format(d, tod.read_flags(detector=d, n=5)))

# Store some data in the cache.  The "cache" member variable looks like a dictionary of
# numpy arrays, but the memory used is allocated in C, so that we can actually clear
# these buffers when needed.

for d in detnames:
    processed = tod.read(detector=d)
    processed /= 2.0
    # By convention, we usually name buffers in the cache by <prefix>_<detector>
    tod.cache.put("processed_{}".format(d), processed)
print("TOD cache now contains {} bytes".format(tod.cache.report(silent=True)))



In [ ]:
if nersc_host is not None:
    %srun -N 1 -C knl -n 32 -c 2 --cpu_bind=cores -t 00:03:00 python intro.py